## hate speech classification demo

In this notebook we will be testing out our model with some random sample tweets. You'll need to [download our model](https://drive.google.com/drive/folders/1UE1MiiNbXWgalJ1UIDW7mTLsSH15UIfX?usp=sharing) and place it in the current directory.

## installation

In [1]:
%pip install --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
%pip install torch
%pip install transformers

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /private/var/folders/j_/ygb_gxx970b1bfpk3v12gc980000gp/T/pip-install-qoszy9rc/twint_54caad899ddc420a9dc12a3334587eb1
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.5/libexec/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.5/libexec/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.5/libexec/bin/python3 -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


## imports

In [1]:
import torch
import transformers
import twint
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, AdamW, DistilBertConfig
import nest_asyncio
nest_asyncio.apply()

Let's load our model and tokenizer.

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
model = torch.load('model.bin', map_location=device)

In [4]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Now, let's fetch some tweets and clean them up.

In [6]:
c = twint.Config()

c.Custom["tweet"] = ["id"]
c.Custom["user"] = ["bio"]
c.Limit = 10
c.Store_csv = True
c.Output = "none"

twint.run.Search(c)

CRITICAL:root:twint.run:Twint:Feed:noData'globalObjects'
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noData'globalObjects'
sleeping for 8.0 secs


KeyboardInterrupt: 

In [8]:
c = twint.Config()

c.Username = "noneprivacy"
c.Custom["tweet"] = ["id"]
c.Custom["user"] = ["bio"]
c.Limit = 10
c.Store_csv = True
c.Output = "none"

twint.run.Search(c)

1381958323986124801 2021-04-13 08:11:54 -0500 <noneprivacy> The FB, Linkedin and Clubhouse "leaks" are not actual leaks  Then, if we want to discuss if being able to get that amount of data is wrong or not, let's chat but it's a totally different topic
1380461578257104896 2021-04-09 05:04:22 -0500 <noneprivacy> @Cloudflare updated their infrastructure IPs list, make the following changes to your iptables rules  Remove:  104.16.0.0/12   Add:  104.16.0.0/13 104.24.0.0/14
1380109111581364225 2021-04-08 05:43:48 -0500 <noneprivacy> @Zewensec @Ginger__T Well, keeping the analogy, I hope that you put the X on your map yourself In my personal vision, it's like your goal
1380106045251588098 2021-04-08 05:31:36 -0500 <noneprivacy> @Ginger__T Aaah damn, felt something was missing and that's it!
1380091641898360832 2021-04-08 04:34:22 -0500 <noneprivacy> #ELI5 #OSINT  OSINT is like doing a treasure hunt, but you need to make your own map.
1379695877800005633 2021-04-07 02:21:45 -0500 <noneprivacy